<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0918T5_demo_filling_blank_question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T5 による，文章穴埋め問題
- date: 2022_0918
- author: 浅川伸一
- filename: `2022_0918T5_demo_filling_blank_question.ipynb`

## 使用上の注意

* `[CLS]` を使用すること。`[CLS]` はモデルの学習時に使用されるため，マスクされたトークンを予測するには，必ず文の前に `[CLS]` トークンを追加して，モデルが正しく符号化できるようにする。
* トークン化した後に `[MASK]` を使用する。入力文字列に直接 [MASK] を入力するのと，トークン化した後に `[MASK]` で置き換えるのとでは、トークンの並びが異なるので，予測結果も異なる。
トークン化後に `[MASK]` を使用する方がよい。
これは，モデルがどのように事前学習されたかと一致するためである。
しかし Huggingface Inference API は入力文字列に `[MASK]` をタイプすることしかサポートしておらず，よりロバストな予測を生成することができない。
* `position_ids` を明示的に引数として与える。
`Roberta` モデルでは，モデルに対して `position_ids` が与えられない場合 Huggingface 版の transformers は自動的に `position_ids` を構築する。だが， 0 ではなく `padding_idx` から作成する。これでは `rinna/japanese-roberta-base` では期待通りに動作しない。
なぜなら対応するトークナイザーの `padding_idx` が 0 ではないからである。
そのため，必ず自分で `position_ids` を `constrcut` して，位置 ID 0 から開始するようにすること。

In [ ]:
%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'
try:
    import bit
except ImportError:
    !pip install --upgrade 'fugashi[ipadic]' > /dev/null 2>&1
    !pip install --upgrade 'fugashi[unidic]' > /dev/null 2>&1
    !python -m unidic download
    !pip install --upgrade ipadic > /dev/null 2>&1
    !pip install --upgrade sentencepiece > /dev/null 2>&1
    !pip install --upgrade transformers > /dev/null 2>&1
    !pip install --upgrade termcolor > /dev/null 2>&1
    !pip install --upgrade jaconv  > /dev/null 2>&1      
    !pip install ipynbname --upgrade > /dev/null 2>&1
    !git clone https://github.com/ShinAsakawa/bit.git
    import bit

isColab = bit.isColab
HOME = bit.HOME

In [ ]:
import torch
from transformers import T5Tokenizer, RobertaForMaskedLM

# load tokenizer
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-roberta-base")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

# load model
model = RobertaForMaskedLM.from_pretrained("rinna/japanese-roberta-base")
model = model.eval()

# 入力文
text = "ぐっすり寝たので、気持ちがいい。"

# prepend [CLS]
text = "[CLS]" + text
print(f'text:{text}')

In [ ]:
# tokenize
tokens = tokenizer.tokenize(text)
print(f'tokens:{tokens}')  
# tokens:['[CLS]', '▁', 'ぐ', 'っ', 'すり', '寝', 'た', 'の', 'で', '、', '気持ち', 'が', 'いい', '。']

In [ ]:
# mask a token; masked_idx の index をマスクする
masked_idx = 2
tokens[masked_idx] = tokenizer.mask_token
print(f'tokens:{tokens}')  
# tokens:['[CLS]', '▁', '[MASK]', 'っ', 'すり', '寝', 'た', 'の', 'で', '、', '気持ち', 'が', 'いい', '。']

In [ ]:
# convert to ids
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f'token_ids:{token_ids}')
# token_ids:[4, 9, 6, 1315, 14073, 4518, 40, 10, 19, 7, 8053, 12, 2505, 8]
print(f'トークン ID をもう一度トークンに戻すと:\n{tokenizer.convert_ids_to_tokens(token_ids)}')
# ['[CLS]', '▁', '[MASK]', 'っ', 'すり', '寝', 'た', 'の', 'で', '、', '気持ち', 'が', 'いい', '。']

In [ ]:
# convert to tensor
token_tensor = torch.LongTensor([token_ids])

# position ids を明示的に与える
position_ids = list(range(0, token_tensor.size(1)))
print(f'position_ids:{position_ids}')  
# position_ids:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
position_id_tensor = torch.LongTensor([position_ids])  # torch.LongTensor とは倍精度整数

# マスクに対応する上位 10 候補を得る
with torch.no_grad():
    outputs = model(input_ids=token_tensor, position_ids=position_id_tensor)
    predictions = outputs[0][0, masked_idx].topk(10)

for i, index_t in enumerate(predictions.indices):
    index = index_t.item()
    token = tokenizer.convert_ids_to_tokens([index])[0]
    print(i, token)

In [ ]:
# 入力文
text = "ぐっすり寝たので、気持ちがいい。"

# prepend [CLS]
text = "[CLS]" + text
print(f'text:{text}')

# tokenize
tokens = tokenizer.tokenize(text)

# convert to tensor
token_tensor = torch.LongTensor([token_ids])

print(f'tokens:{tokens}')  
# tokens:['[CLS]', '▁', 'ぐ', 'っ', 'すり', '寝', 'た', 'の', 'で', '、', '気持ち', 'が', 'いい', '。']

# mask a token; masked_idx の index をマスクする
for masked_idx in [2, 3, 4]:
    tokens[masked_idx] = tokenizer.mask_token
print(f'tokens:{tokens}')  
# tokens:['[CLS]', '▁', '[MASK]', '[MASK]', '[MASK]', '寝', 'た', 'の', 'で', '、', '気持ち', 'が', 'いい', '。']

# position ids を明示的に与える
position_ids = list(range(0, token_tensor.size(1)))
print(f'position_ids:{position_ids}')  
# position_ids:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
position_id_tensor = torch.LongTensor([position_ids])

# マスクに対応する上位 10 候補を得る
with torch.no_grad():
    outputs = model(input_ids=token_tensor, position_ids=position_id_tensor)
    for masked_idx in [2,3,4]:
        predictions = outputs[0][0, masked_idx].topk(10)
        print(f'masked_idx:{masked_idx}')
        for i, index_t in enumerate(predictions.indices):
            index = index_t.item()
            token = tokenizer.convert_ids_to_tokens([index])[0]
            print(f'\t{i}:{token}', end=" ")
        print()            

In [ ]:
def gpt_predict(
    full_text:str, 
    blank:str,
    top_k:int=10,
    model=model,
    ):
    model = model.eval()                
    
    text = '[CLS]' + full_text
    tokens = tokenizer.tokenize(text)
    
    # convert to ids
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    
    masked_tokens = tokenizer.tokenize(blank)[1:]
    masked_pos = []
    for masked_token in masked_tokens:
        #print(f'masked_token:{masked_token}')
        try:
            if tokens.index(masked_token):
                pos = tokens.index(masked_token)
        except ValueError:
            pos = -1
        if pos != -1:
            token_ids[pos] = tokenizer.mask_token_id
            masked_pos.append(pos)

    print(f'トークン ID をもう一度トークンに戻す:{tokenizer.convert_ids_to_tokens(token_ids)}')
            
    # convert to tensor
    token_tensor = torch.LongTensor([token_ids])

    position_ids = list(range(0, token_tensor.size(1)))
    position_id_tensor = torch.LongTensor([position_ids])
    
    # マスクに対応する上位 top_k 候補を得る
    with torch.no_grad():
        outputs = model(input_ids=token_tensor, 
                        position_ids=position_id_tensor)
    for _masked_pos in masked_pos:
        predictions = outputs[0][0, _masked_pos].topk(10)
        print(f'_masked_pos:{_masked_pos}')
        for i, index_t in enumerate(predictions.indices):
            index = index_t.item()
            token = tokenizer.convert_ids_to_tokens([index])[0]
            print(f'\t{i}:{token}', end=" ")
        print()                            
    #print(tokens)

gpt_predict(
    full_text='このパソコンは誰でも使えますが，コピーは有料です', 
    blank='有料',
    model=model)

gpt_predict(
    full_text='次々に新しいゲームが作られる。', 
    blank='次々に',
    model=model)
